In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms

from data import data_loaders, AddGaussianNoise
from main import Trainer, make_predictions
from models import resnet18, LabelSmoothingCrossEntropy

SEED = 42

torch.manual_seed(SEED);

results = dict()

In [ ]:
# Data parameters
num_classes = 43
batch_size = 128
data_transforms = None  # Default

train_loader, val_loader = data_loaders(batch_size, data_transforms)

In [ ]:
# Training parameters
resnet_variant = None  # Default
epochs = 25
criterion = nn.CrossEntropyLoss()
lr = 0.001

model = resnet18(variant=resnet_variant, num_classes=num_classes)

trainer = Trainer(model, train_loader, val_loader, criterion, lr)
trainer.run(epochs)

results["config_1"] = {
    "description": "Default ResNet, no data augmentation",
    "train_loss": trainer.train_loss,
    "test_loss": trainer.test_loss,
    "test_accuracy": trainer.test_accuracy
}

In [ ]:
# Data parameters
num_classes = 43
batch_size = 128
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop((32, 32)),
    transforms.ToTensor(),
    AddGaussianNoise(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), (0.2672, 0.2564, 0.2629)),
])

train_loader, val_loader = data_loaders(batch_size, data_transforms)

In [ ]:
# Training parameters
resnet_variant = None  # Default
epochs = 25
criterion = nn.CrossEntropyLoss()
lr = 0.001

model = resnet18(variant=resnet_variant, num_classes=num_classes)

trainer = Trainer(model, train_loader, val_loader, criterion, lr)
trainer.run(epochs)

results["config_2"] = {
    "description": "Default ResNet, with data augmentation",
    "train_loss": trainer.train_loss,
    "test_loss": trainer.test_loss,
    "test_accuracy": trainer.test_accuracy
}

In [ ]:
# Training parameters
resnet_variant = "C"
epochs = 25
criterion = nn.CrossEntropyLoss()
lr = 0.001

model = resnet18(variant=resnet_variant, num_classes=num_classes)

trainer = Trainer(model, train_loader, val_loader, criterion, lr)
trainer.run(epochs)

results["config_3"] = {
    "description": "ResNet-C, with data augmentation",
    "train_loss": trainer.train_loss,
    "test_loss": trainer.test_loss,
    "test_accuracy": trainer.test_accuracy
}

In [ ]:
# Training parameters
resnet_variant = "D"
epochs = 25
criterion = nn.CrossEntropyLoss()
lr = 0.001

model = resnet18(variant=resnet_variant, num_classes=num_classes)

trainer = Trainer(model, train_loader, val_loader, criterion, lr)
trainer.run(epochs)

results["config_4"] = {
    "description": "ResNet-D, with data augmentation",
    "train_loss": trainer.train_loss,
    "test_loss": trainer.test_loss,
    "test_accuracy": trainer.test_accuracy
}

In [ ]:
make_predictions(trainer.model, trainer.device)